In [ ]:
import os
import scipy
import numpy as np
import trimesh
from trimesh import viewer
from trimesh.transformations import translation_matrix, rotation_matrix, concatenate_matrices
from scipy.spatial.transform import Rotation
from scipy.spatial import cKDTree

np.set_printoptions(precision=3, suppress=True, linewidth=np.inf)


In [ ]:
mesh1 = trimesh.exchange.load.load('wolf1.ply')
mesh2 = mesh1.copy()

# slice mesh1
mesh1 = mesh1.slice_plane(plane_normal=[0, -1, 0], plane_origin=[10, 10, 10])
# apply an arbitrary transformation to mesh1
T = translation_matrix([-11, 0, 11])
xaxis, yaxis, zaxis = [1, 0, 0], [0, 1, 0], [0, 0, 1]
Rx = rotation_matrix(0, xaxis)
Ry = rotation_matrix(-np.pi/6, yaxis)
Rz = rotation_matrix(0, zaxis)
M = concatenate_matrices(T, Rx, Ry, Rz)
mesh1.apply_transform(M)

# write scene to html
if os.path.exists("scene.html"):
    os.remove("scene.html")
html = viewer.notebook.scene_to_html((trimesh.util.concatenate([mesh1, mesh2])).scene())
with open("scene.html", "w") as file:
    file.write(html)


In [ ]:
def register(source, target, w1=0.1, w2=1.0, max_iter=100):
    # resulting transformation is (R, t)
    transform = (np.eye(3), np.zeros(3))
    x = source.vertices
    y = target.vertices
    N = len(x)
    z = x.copy()
    tree = cKDTree(y)
    for iter in range(max_iter):
        # applies transformation to original x
        x_t = x.dot(transform[0].T) + transform[1]
        # build a matrix equivalent to cross product
        X_t = np.stack([
            np.stack((np.zeros(N), -x_t[:, 2], x_t[:, 1]), axis=1),
            np.stack((x_t[:, 2], np.zeros(N), -x_t[:, 0]), axis=1),
            np.stack((-x_t[:, 1], x_t[:, 0], np.zeros(N)), axis=1)
        ], axis=1)
        # build blocks of sparse coefficient matrix A
        A_rr = -np.einsum('abc,acd->bd', X_t, X_t)
        A_tr = np.sum(X_t, axis=0)
        A_zir = -np.concatenate([X_ti for X_ti in X_t], axis=1)
        A_rt = -np.sum(X_t, axis=0)
        A_tt = N * np.eye(3)
        A_zit = -np.tile(np.eye(3), (1, N))
        A_rzi = np.concatenate([X_ti for X_ti in X_t], axis=0)
        A_tzi = -np.tile(np.eye(3), (N, 1))
        A_zizj = scipy.sparse.eye(3 * N) * (w1 / w2 + 1)
        A_rr = scipy.sparse.csr_matrix(A_rr)
        A_tr = scipy.sparse.csr_matrix(A_tr)
        A_zir = scipy.sparse.csr_matrix(A_zir)
        A_rt = scipy.sparse.csr_matrix(A_rt)
        A_tt = scipy.sparse.csr_matrix(A_tt)
        A_zit = scipy.sparse.csr_matrix(A_zit)
        A_rzi = scipy.sparse.csr_matrix(A_rzi)
        A_tzi = scipy.sparse.csr_matrix(A_tzi)
        A_zizj = scipy.sparse.csr_matrix(A_zizj)
        # assemble coefficient matrix A
        A = scipy.sparse.bmat([[A_rr, A_tr, A_zir],
                               [A_rt, A_tt, A_zit],
                               [A_rzi, A_tzi, A_zizj]], format='csr')
        # build blocks of result vector b
        b_r = np.zeros(3)
        b_t = -np.sum(x_t, axis=0)
        _, PI = tree.query(z, k=1)
        b_zi = (w1 / w2 * y[PI] + x_t).flatten()
        # assemble result vector b
        b = np.concatenate([b_r, b_t, b_zi])
        # solve system of linear equations Ax = b using conjugate gradient method
        solution, info = scipy.sparse.linalg.cg(A, b, tol=1e-10, maxiter=1000)
        # ensure convergence
        assert info == 0
        # update resulting transformation
        r = solution[:3]
        t = solution[3:6]
        z = solution[6:].reshape(-1, 3)
        R = Rotation.from_euler('xyz', r).as_matrix()
        # ensure that R is a rotation matrix
        assert np.isclose(np.linalg.det(R), 1.0)
        transform = (np.dot(R, transform[0]), t + transform[1])
        print(iter, '\n', transform, '\n')
    return transform, z


In [ ]:
(R, t), z = register(mesh1, mesh2, max_iter=100)
print('ground truth\n', M[:3, :3], M[:3, 3])
trans_mesh = trimesh.base.Trimesh(mesh1.vertices.dot(R.T) + t, mesh1.faces) # z is also interesting

trans_mesh.visual.vertex_colors = trimesh.visual.random_color()

if os.path.exists("scene.html"):
    os.remove("scene.html")
html = viewer.notebook.scene_to_html(trimesh.util.concatenate([trans_mesh, mesh2]).scene())
with open("scene.html", "w") as file:
    file.write(html)
